In [1]:
# First we will need to install and load a variety of packages to prep the sequence data for training. 

import pandas as pd
import subprocess
import shutil
from collections import defaultdict
from pathlib import Path
import numpy as np
import itertools as it
import random
from sklearn.model_selection import train_test_split

In [2]:
# import training data 
data_df = pd.read_excel("./All_LRR_PRR_ligand_data.xlsx")
#data_df = data_df[~data_df["Literature_Data"].isin(["01_Steinbrenner_2020", "02_Snoeck_2022","22_Kim_2020"])]

# this will pull parsed receptor sequences from the fasta files
# subset data_df to only include Receptor Name and Epitope columns without any duplicates
receptor_ligand_pairs = data_df[["Plant species", "Receptor", "Ligand","Ligand Sequence", 'Immunogenicity']].drop_duplicates()

# creating a dictionary of ligand to plant receptor for aiding in mapping ectodomain sequence to correct receptor
receptor_ligand_pairs['Receptor Name'] = receptor_ligand_pairs['Plant species'] + '|' + receptor_ligand_pairs['Receptor']
ligand_to_plant_receptor = {
    row['Ligand']: f">{row['Plant species']}|{row['Receptor']}"
    for _, row in receptor_ligand_pairs.iterrows()
}
#print(ligand_to_plant_receptor)


# input the fasta file called receptor_ectodomains.fasta and read each line such that it pulls the receptor sequence and name and save it to a dictionary called receptor_name_to_seq   
receptor_name_to_seq = {}       
with open("./in_data/receptor_ectodomains.fasta", "r") as f:
    lines = f.read().splitlines()
    for line in lines:
        if line.startswith('>'):
            receptor_name = line[1:]
            # Remove the final | and number range if present
            if '|' in receptor_name:
                receptor_name = receptor_name.rsplit('|', 1)[0]
            receptor_name_to_seq[receptor_name] = ''
        else:
            receptor_name_to_seq[receptor_name] += line
#print(receptor_name_to_seq)


ImportError: Missing optional dependency 'openpyxl'.  Use pip or conda to install openpyxl.

In [93]:
#receptor_ligand_pairs['Receptor Name'] = receptor_ligand_pairs["Ligand"].map(ligand_to_plant_receptor)
receptor_ligand_pairs['Receptor Sequence'] = receptor_ligand_pairs["Receptor Name"].map(receptor_name_to_seq)
#print(receptor_ligand_pairs)


# help me split receptor_ligand_pairs into train and test sets based on the immunogenicity column and sequence variation found in Ligand Sequence column
stratify_array = [(epitope, outcome) for epitope, outcome in zip(receptor_ligand_pairs['Ligand Sequence'], receptor_ligand_pairs['Immunogenicity'])]
print(stratify_array)

# Create a function to handle rare cases
def split_with_rare_handling(df, min_samples=2):
    # Identify rare combinations
    combinations = df.groupby(['Ligand Sequence', 'Immunogenicity']).size()
    rare_combinations = combinations[combinations < min_samples].index
    
    # Split the data into rare and common cases
    rare_mask = df.apply(lambda x: (x['Ligand Sequence'], x['Immunogenicity']) in rare_combinations, axis=1)
    rare_cases = df[rare_mask]
    common_cases = df[~rare_mask]
    
    # Split common cases with stratification
    if len(common_cases) > 0:
        stratify_array = [(epitope, outcome) for epitope, outcome in 
                         zip(common_cases['Ligand Sequence'], common_cases['Immunogenicity'])]
        common_train, common_test = train_test_split(
            common_cases, test_size=0.2, random_state=42, stratify=stratify_array
        )
    else:
        common_train, common_test = pd.DataFrame(), pd.DataFrame()
    
    # Split rare cases without stratification
    if len(rare_cases) > 0:
        rare_train, rare_test = train_test_split(
            rare_cases, test_size=0.2, random_state=42
        )
    else:
        rare_train, rare_test = pd.DataFrame(), pd.DataFrame()
    
    # Combine the results
    train_df = pd.concat([common_train, rare_train])
    test_df = pd.concat([common_test, rare_test])
    
    return train_df, test_df

# Use the function
train_df, test_df = split_with_rare_handling(receptor_ligand_pairs)
#print(train_df)
print(test_df)

train_df.to_csv("./split_training_data/train_stratify.csv", index=False)
test_df.to_csv("./split_training_data/test_stratify.csv", index=False)


[('DVTAGAEVWNQPVRGFKVYEQTEMT', 'Immunogenic'), ('DVTAGAEVANQPVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQAVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQPVRGFKVAEQTEMT', 'Immunogenic'), ('DVTAGAEVWNQPVRGFKVYEQTEMT', 'Immunogenic'), ('DVTAGAEVANQPVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQAVRGFKVYEQTEMT', 'Weakly Immunogenic'), ('DVTAGAEVWNQPVRGFKVAEQTEMT', 'Immunogenic'), ('DVTAGAEVWNQPVRGFKVYEQTEMT', 'Weakly Immunogenic'), ('DVTAGAEVANQPVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQAVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQPVRGFKVAEQTEMT', 'Weakly Immunogenic'), ('DVTAGAEVWNQPVRGFKVYEQTEMT', 'Immunogenic'), ('DVTAGAEVANQPVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQAVRGFKVYEQTEMT', 'Immunogenic'), ('DVTAGAEVWNQPVRGFKVAEQTEMT', 'Immunogenic'), ('DVTAGAEVWNQPVRGFKVYEQTEMT', 'Immunogenic'), ('DVTAGAEVANQPVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQAVRGFKVYEQTEMT', 'Non-Immunogenic'), ('DVTAGAEVWNQPVRGFKVAEQTEMT', 'Immunogenic'), ('DVTAGAEVWNQPVRGFKVYEQTEM

In [ ]:
#for outcome in receptor_ligand_pairs['Immunogenicity'].unique():
#    with open(f"./split_training_data/sequences_{outcome.replace(' ', '-')}.fasta", "w") as f:
#        outcome_df = receptor_ligand_pairs[receptor_ligand_pairs['Immunogenicity'] == outcome]
#        for seq in outcome_df['Ligand Sequence']:
#            f.write(f">{seq}\n")
#               f.write(f"{seq}\n")
    
    #subprocess.run(f"mmseqs easy-cluster ../out_data/fastas/sequences_{outcome.replace(' ', '-')}.fasta cluster50_{outcome.replace(' ', '-')} tmp --min-seq-id 0.5 -c 0.8 --cov-mode 1", shell=True, check=True)
    #shutil.rmtree("tmp")
    #subprocess.run(f"mv cluster50_{outcome.replace(' ', '-')}* ../out_data/mmseqs/", shell=True, check=True)

# for epitope in data_df['Epitope'].unique():
#     with open(f"../out_data/fastas/sequences_{epitope}.fasta", "w") as f:
#         epitope_df = data_df[data_df['Epitope'] == epitope]
#         for seq in epitope_df['Sequence']:
#             f.write(f">{seq}\n")
#             f.write(f"{seq}\n")
    
#     subprocess.run(f"mmseqs easy-cluster ../out_data/fastas/sequences_{epitope}.fasta cluster20_{epitope} tmp --min-seq-id 0.2 -c 0.8 --cov-mode 1", shell=True, check=True)
#     shutil.rmtree("tmp")
#     subprocess.run(f"mv cluster20_{epitope}* ../out_data/mmseqs/", shell=True, check=True)

# with open(f"../out_data/fastas/sequences.fasta", "w") as f:
#     for seq in data_df['Sequence']:
#         f.write(f">{seq}\n")
#         f.write(f"{seq}\n")


# subprocess.run("mmseqs easy-cluster ../out_data/fastas/sequences.fasta cluster20 tmp --min-seq-id 0.2 -c 0.8 --cov-mode 1", shell=True, check=True)
# shutil.rmtree("tmp")
# subprocess.run("mv cluster20* ../out_data/mmseqs/", shell=True, check=True)